# Projects markdown generator for academicpages

Takes a TSV of projects with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, slides_url, video_url, excerpt, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat projects.json

[
    {
        "title": "Analyzing and Mitigating Dataset Artifacts",
        "contributors": "Md Mesbahur Rahman",
        "type": "Academic",
        "project_date": "2022-12-11", 
        "url_slug": "nlp-ut-mscs-nlp-project", 
        "teaser_url": "sample-number-vs-f1.png",
        "report_url": "https://mmrahman-utexas.github.io/files/UTCS_NLP_Analyzing_and_Mitigating_Dataset_Artifacts.pdf",
        "code_url": "",
        "excerpt": "In NLP research arena Benchmark datasets are often used to compare the performance of different SOTA models. But a high held-out accuracy
        measure neither conveys the whole story about a model's strengths and weaknesses nor it can guarantee that the model has meaningfully solved the dataset. The model can just learn some spurious correlation in the dataset and can still achieve some high accuracy. This phenomenon is known
        as Dataset Artifacts and in this project, we tried to identify some cases of it for the ELECTRA-small (Clark et a

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
projects = pd.read_json("projects.json")
projects

,title,contributors,type,project_date,url_slug,teaser_url,report_url,code_url,excerpt,my_contribution
0,Analyzing and Mitigating Dataset Artifacts,Md Mesbahur Rahman,Academic,2022-12-11,nlp-ut-mscs-nlp-project,sample-number-vs-f1.png,https://mmrahman-utexas.github.io/files/UTCS_N...,,In NLP research arena Benchmark datasets are o...,Trained ELECTRA-small model on the SQuAD data...
1,Unsupervised Anomaly Detection Using Convoluti...,Md Mesbahur Rahman,Academic,2020-05-31,Unsupervised-Anomaly-Detection-Using-Convoluti...,anomaly-digit.png,https://github.com/mmrahman-utexas/Unsupervise...,,Anomaly detection is a very common and importa...,Built an unsupervised dataset from a supervise...
2,Image Caption Generation using CNN LSTM Encode...,Md Mesbahur Rahman,Academic,2020-05-24,Image-Caption-Generation-using-CNN-LSTM-Encode...,image-captioning-infer2.png,,https://github.com/mmrahman-utexas/Image_Capti...,Image caption generation is a widely used appl...,Pre-processed the images in the MS COCO Datase...
3,Facial Keypoint Detection using CNN Haar Casca...,Md Mesbahur Rahman,Academic,2017-09-08,Facial-Keypoint-Detection-using-CNN-Haar-Casca...,facial_leypoint_infer.png,,https://github.com/mmrahman-utexas/Facial_Keyp...,Facial keypoint detection is an important exam...,Defined and trained a CNN on facial keypoint d...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in projects.iterrows():
    
    md_filename = str(item.project_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.project_date) + "-" + item.url_slug 
    year = item.project_date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: projects" + "\n"
    md += f'urlslug: "{item.url_slug }"\n'
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Project"\n'
    md += "permalink: /projects/" + html_filename + "\n"
    if len(str(item.contributors)) > 3:
        md += 'contributors: "' + item.contributors + '"\n'
    if len(str(item.my_contribution)) > 3:
        md += 'contribution: "' + item.my_contribution + '"\n'
    md += "date: " + str(item.project_date) + "\n"
    if len(str(item.teaser_url)) > 5:
        md += "teaserurl: '" + item.teaser_url + "'\n"
    report_url = None
    if len(str(item.report_url)) > 5:
        report_url = str(item.report_url)
        md += "reporturl: '" + item.report_url + "'\n"
    code_url = None
    if len(str(item.code_url)) > 5:
        code_url = str(item.code_url)
        md += "codeurl: '" + item.code_url + "'\n"
    if len(str(item.excerpt)) > 5:
        md += "excerpt: '" + html_escape(item.excerpt) + "'\n"
           
    md += "---\n"

    more_info = []
    if code_url is not None:
        more_info.append(f'[[Code]({code_url})]')
    if report_url is not None:
        more_info.append(f'[[Technical report]({report_url})]')
    if len(item.contributors) > 0:
        md += f"\n{item.contributors}\n"
    if len(str(item.excerpt)) > 4:
        md += f"\n**Description:**\n{html_escape(item.excerpt)}\n"
    if len(str(item.my_contribution)) > 4:
        md += f"\n**My contribution:**\n{html_escape(item.my_contribution)}\n"
    if len(more_info) > 0:
        md += f"\n**Resources:** {' '.join(more_info)}\n"
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_projects/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_projects

2017-09-08-Facial-Keypoint-Detection-using-CNN-Haar-Cascade-Classifier.md
2020-05-24-Image-Caption-Generation-using-CNN-LSTM-Encoder-Decoder.md
2020-05-31-Unsupervised-Anomaly-Detection-Using-Convolutional-Autoencoder.md
2022-12-11-nlp-ut-mscs-nlp-project.md


In [8]:
!cat ../_projects/2022-12-11-nlp-ut-mscs-nlp-project.md

---
title: "Analyzing and Mitigating Dataset Artifacts"
collection: projects
urlslug: "nlp-ut-mscs-nlp-project"
type: "Academic"
permalink: /projects/2022-12-11-nlp-ut-mscs-nlp-project
contributors: "Md Mesbahur Rahman"
contribution: "Trained ELECTRA-small model  on the SQuAD dataset. Then we generated predictions for the respective dataset of Checklist sets and Adversarial SQuAD from this model using our own scripts. Then we used Checklist and Adversarial framework to identify some of the artifacts in the model’s learning. Implemented Inoculation by fine-tuning mwthod for mitigating dataset artifacts by taking our original Electra-small model training on the training set of the SQuAD dataset and fine-tuning it on a small subset sampled from the training set of the 'Adversarial SQuAD' dataset. Then we evaluated dataset artifacts of this finetuned model using Checklist sets and Adversarial SQuAD and caompared with the original result."
date: 2022-12-11
teaserurl: 'sample-number-vs-f1.pn